# BERT on IMDB Sentiment Analysis

这个文件在IMDB上进行情感分类，熟悉transformers库的使用。本notebook中不再使用torchtext。测试以下模型的表现：

* BERT(FT)

目前的进度：

* 未完成

问题：

参考：

## Requirement

## Import

In [1]:
import torch
import numpy as np
import random
from torch import nn,optim
from sklearn import metrics
import torch.nn.functional as F
from datasets import list_datasets, load_dataset, list_metrics, load_metric

use_cuda=torch.cuda.is_available()
device=torch.device("cuda" if use_cuda else "cpu")

SEED = 1234
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if use_cuda:
    torch.cuda.manual_seed(SEED)

## 可供调整的参数

In [2]:
bs=128
d_hidden=256
d_output=2
n_layers=2
bidirectional=True
dropout=0.25
model_name='1'
max_epochs=1
require_improvement=3

## Data

**首先为了使用BERT，我们需要使用配套的tokennizer**

In [20]:
imdb_dataset = load_dataset('imdb')
test_data=imdb_dataset['test']
trainandval=imdb_dataset['train'].train_test_split(0.2)
train_data=trainandval['train']
val_data=trainandval['test']

Reusing dataset imdb (C:\Users\hp\.cache\huggingface\datasets\imdb\plain_text\1.0.0\90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
Loading cached split indices for dataset at C:\Users\hp\.cache\huggingface\datasets\imdb\plain_text\1.0.0\90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3\cache-e1cf38b3bb8ee766.arrow and C:\Users\hp\.cache\huggingface\datasets\imdb\plain_text\1.0.0\90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3\cache-13f73fd60743e26c.arrow


In [34]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
dataset = val_data.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)

In [42]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [38]:
dataset.set_format(type='torch', columns=['input_ids','attention_mask', 'label'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)

## Model

In [43]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

对于BertModel来说，它的返回由四个部分组成：
* last_hidden_state ：（batch,seq,hidden_size）
整个输入的句子每一个token的隐层输出，也是我们这里将要用到的，可以将它作为embedding的替代
* pooler_output：（batch,hidden_size）
输入句子第一个token的最高隐层。也就是[CLS]标记提取到的最终的句对级别的抽象信息。对于BERT的预训练来说，这个隐层信息将作为Next Sentence prediction任务的输入。然而我们这里将不会用到它，因为它对于情感分析来说效果不是很好。
This output is usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence.
* hidden_states：
一个元组，里面每个元素都是(batch,seq,hidden_size) 大小的FloatTensor，分别代表每一层的隐层和初始embedding的和
* attentions:
一个元组，里面每个元素都是(batch,num_heads,seq,seq) 大小的FloatTensor，分别表示每一层的自注意力分数。

在本实验中，我们仅用BERT作为一个特征抽取器，固定参数，不去进行微调。

In [35]:
class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        #固定参数，不对BERT进行微调
        for p in self.parameters():
            p.requires_grad = False
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            #固定参数，不对BERT进行微调
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, hidden_dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output
    
model = BERTGRUSentiment(bert,
                         d_hidden,
                         d_output,
                         n_layers,
                         bidirectional,
                         dropout)

In [36]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


测试一下能否跑通

In [37]:
with torch.no_grad():
    criterion = nn.CrossEntropyLoss()
    if use_cuda:
        model.to(device)
        criterion.to(device)
    for batch in train_iterator:
        #print(batch.text.shape)
        #print(batch.label)
        logits=model(batch.text)
        print(logits.shape)
        criterion(logits,batch.label)
        break

torch.Size([4, 2])


## Training

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [39]:
def train(model, train_iter, dev_iter, test_iter):
    model.to(device)
    model.train()
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()
    #criterion = nn.BCEWithLogitsLoss()
    if use_cuda:
        criterion.cuda()

    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    dev_best_loss = float('inf')
    last_improve = 0  # 记录上次验证集loss下降的batch数
    #writer = SummaryWriter(log_dir=config.log_path + '/' + time.strftime('%m-%d.%H.%M', time.localtime())+'_'+which_data+'_'+which_model+'_'+which_task+'_'+exp_number)
    
    for epoch in range(max_epochs):
        start_time = time.time()
        train_loss=0
        train_correct=0
        # scheduler.step() # 学习率衰减
        for i, batch in enumerate(train_iter):
            optimizer.zero_grad()
            x,l=batch.text
            y=batch.label
            outputs = model(x,l)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            #训练集的准确率
            preds = torch.max(outputs, 1)[1]
            #preds = torch.round(torch.sigmoid(outputs))
            train_correct+=(y==preds).sum()
            train_loss+=loss.item()
        train_loss/=len(train_iterator)   #train_loss
        train_acc=train_correct/len(train_iterator.dataset)   #train_acc
            
        #验证集
        dev_acc, dev_loss = evaluate(model, dev_iter)
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        if dev_loss < dev_best_loss:
            dev_best_loss = dev_loss
            improve = '*'
            last_improve=epoch
            torch.save(model.state_dict(),model_name+'.pth')
        else:
            improve = ''
        msg = 'Epoch: {0:>3},  Epoch Time: {1}m {2}s,  Train Loss: {3:>5.2},  Train Acc: {4:>6.2%},  Val Loss: {5:>5.2},  Val Acc: {6:>6.2%} {7}'
        print(msg.format(epoch+1,epoch_mins, epoch_secs,train_loss, train_acc, dev_loss, dev_acc, improve))
        #writer.add_scalar("loss/train", loss.item(), total_batch)
        #writer.add_scalar("loss/dev", dev_loss, total_batch)
        #writer.add_scalar("acc/train", train_acc, total_batch)
        #writer.add_scalar("acc/dev", dev_acc, total_batch)

        if epoch - last_improve > require_improvement:
            # 验证集loss超过1epoch没下降，结束训练
            print("No optimization for a long time, auto-stopping...")
            break
    #writer.close()
    #训练跑完了，使用最佳模型测试
    model.load_state_dict(torch.load(model_name+'.pth'))
    test(model, test_iter)

def evaluate(model, data_iter, test=False):
    model.eval()
    loss_total = 0
    predict_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    with torch.no_grad():
        for batch in data_iter:
            x,l=batch.text
            labels=batch.label
            outputs = model(x,l)
            loss = F.cross_entropy(outputs, labels)
            #loss=criterion(outputs,labels)
            loss_total += loss
            labels = labels.data.cpu().numpy()
            predic = torch.max(outputs, 1)[1].cpu().numpy()
            #predic=torch.round(torch.sigmoid(outputs)).cpu().numpy()
            labels_all = np.append(labels_all, labels)
            predict_all = np.append(predict_all, predic)
    model.train()
    acc = metrics.accuracy_score(labels_all, predict_all)
    
    if test:
        report = metrics.classification_report(labels_all, predict_all, labels=[0,1],target_names=['pos','neg'], digits=4,output_dict=True)
        confusion = metrics.confusion_matrix(labels_all, predict_all)
        return acc, loss_total / len(data_iter), report, confusion
    
    return acc, loss_total / len(data_iter)


def test(model, test_iter):
    test_acc, test_loss, test_report, test_confusion = evaluate(model, test_iter, test=True)
    msg = 'Test Loss: {0:>5.2},  Test Acc: {1:>6.2%}'
    print(msg.format(test_loss, test_acc))
    print("Precision, Recall and F1-Score...")
    print(test_report)
    print("Confusion Matrix...")
    print(test_confusion)

In [ ]:
#original
train(model,train_iterator,valid_iterator,test_iterator)